In [ ]:
import pandas as pd
import glob
import os

In [ ]:
# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "energy_ microinversores")
print(caminho)

In [ ]:
# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

In [ ]:
# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

In [ ]:
# Função para concatenar os DataFrames
def concatenar_dataframes(lista):
    """Concatena todos os DataFrames em um único DataFrame."""
    if not lista:
        print('Nenhum dado para concatenar.')
        return None
    return pd.concat(lista, ignore_index=True)

In [ ]:
# # Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
# def adicionar_id_e_nome_microinversor(df):
#     """
#     Adiciona uma coluna 'ID' no início do DataFrame e uma coluna 'Microinversor' baseada na coluna 'SN'.
#     Associa cada SN a um nome de microinversor único.
#     """
#     # Adicionar a coluna 'ID' com valores sequenciais
#     # df.insert(0, 'ID', range(1, len(df) + 1))

#     if 'SN' not in df.columns and 'Sn' in df.columns:
#         df.rename(columns={'Sn': 'SN'}, inplace=True)

#     if 'SN' not in df.columns:
#         print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
#         return df

#     # Criar um dicionário associando os SNs a nomes de microinversores
#     microinversores_dict = {sn: f"Microinversor_{idx+1}" for idx, sn in enumerate(df['SN'].unique())}

#     # Mapear a coluna SN para os nomes dos microinversores
#     df['Microinversor'] = df['SN'].map(microinversores_dict)

#     return df


# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_id_e_nome_microinversor(df):
    """
    Adiciona uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor fixo, conforme um dicionário predefinido.
    """
    # Verificar e renomear a coluna 'Sn' para 'SN', se necessário
    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Dicionário fixo associando os SNs aos nomes dos microinversores
    microinversores_dict = {
        "106272403152": "Micro_04",
        "106272403916": "Micro_04",
        "106272404964": "Micro_04",
        "116191966583": "Micro_04"
    }

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    return df

In [ ]:
# Função para salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Função principal
def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = adicionar_id_e_nome_microinversor(df_novo)
        display(df_novo.head())
        salvar_dataframe(df_novo, caminho_novo_arquivo)

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
#TODO: Função temporaria
df_temp = pd.read_csv(os.path.join(caminho, 'arquivo.csv'))
df_temp.head()

In [ ]:
valores_unicos = df_temp['SN'].unique()

print(valores_unicos)

In [ ]:
# Substituir os valores na coluna 'SN'
df_temp['SN'] = df_temp['SN'].replace(106272404964, 116191966583)

# Verificar se a substituição foi feita corretamente
print(df_temp['SN'].unique())

In [ ]:
# Contar a frequência dos valores na coluna 'SN'
frequencia_valores = df_temp['SN'].value_counts()

print(frequencia_valores)

In [ ]:

def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Salvar o DataFrame atualizado
caminho_novo_arquivo = os.path.join(caminho, 'arquivo_atualizado.csv')
salvar_dataframe(df_temp, caminho_novo_arquivo)
# Exibir os primeiros registros do DataFrame atualizado
df_temp.head()

In [7]:
import pandas as pd
import glob
import os

# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "energy_microinversores")
print(caminho)

# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

# Função para concatenar os DataFrames
def concatenar_dataframes(lista):
    """Concatena todos os DataFrames em um único DataFrame."""
    if not lista:
        print('Nenhum dado para concatenar.')
        return None
    return pd.concat(lista, ignore_index=True)

# Função para criar uma nova coluna contendo o nome do microinversor baseado no SN
def adicionar_id_e_nome_microinversor(df):
    """
    Adiciona uma coluna 'Microinversor' baseada na coluna 'SN'.
    Associa cada SN a um nome de microinversor fixo, conforme um dicionário predefinido.
    """
    # Verificar e renomear a coluna 'Sn' para 'SN', se necessário
    if 'SN' not in df.columns and 'Sn' in df.columns:
        df.rename(columns={'Sn': 'SN'}, inplace=True)

    if 'SN' not in df.columns:
        print("Erro: A coluna 'SN' não foi encontrada no arquivo.")
        return df

    # Garantir que os valores na coluna 'SN' sejam strings
    df['SN'] = df['SN'].astype(str)

    # Dicionário fixo associando os SNs aos nomes dos microinversores
    microinversores_dict = {
        "106272403152": "Micro_01",
        "106272403916": "Micro_02",
        "106272404964": "Micro_03",
        "116191966583": "Micro_04"
    }

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)

    # Preencher valores não encontrados no dicionário com "Desconhecido"
    df['Microinversor'] = df['Microinversor'].fillna("Desconhecido")

    return df

    # Mapear a coluna SN para os nomes dos microinversores
    df['Microinversor'] = df['SN'].map(microinversores_dict)
    return df  # Adicionado o retorno do DataFrame atualizado

# Função para salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

# Substituir os valores na coluna 'SN'
def substituir_valores_sn(df_temp):
    """Substitui os valores na coluna 'SN'."""
    if 'SN' in df_temp.columns:  # Verificar se a coluna 'SN' existe
        df_temp['SN'] = df_temp['SN'].replace(116191966583, 106272404964)
    else:
        print("Erro: A coluna 'SN' não foi encontrada no DataFrame.")
    return df_temp

# Função para processar o DataFrame
def processar_dataframe(df):
    """
    Realiza as seguintes operações no DataFrame:
    1. Renomeia a coluna 'Energy (kWh)' para 'Energy'.
    2. Converte a coluna 'Date' para o tipo datetime.
    3. Adiciona uma nova coluna 'Year' contendo o ano extraído da coluna 'Date'.
    """
    # Renomear a coluna 'Energy (kWh)' para 'Energy'
    if 'Energy (kWh)' in df.columns:
        df.rename(columns={'Energy (kWh)': 'Energy'}, inplace=True)

    # Converter a coluna 'Date' para o tipo datetime
    if 'Date' in df.columns:
        try:
            df['Date'] = pd.to_datetime(df['Date'])
        except Exception as e:
            print(f"Erro ao converter a coluna 'Date' para datetime: {e}")

    # Adicionar uma nova coluna 'Year' contendo o ano
    if 'Date' in df.columns:
        df['Year'] = df['Date'].dt.year

    return df

# Função principal
def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = processar_dataframe(df_novo)  # Processar o DataFrame
        df_novo = adicionar_id_e_nome_microinversor(df_novo)
        # df_novo = substituir_valores_sn(df_novo)
        display(df_novo.head())
        salvar_dataframe(df_novo, caminho_novo_arquivo)

if __name__ == "__main__":
    main()

files/energy_microinversores
Erro ao ler files/energy_microinversores/chart_csv_108864.csv: Missing column provided to 'parse_dates': 'Date'


,Plant Name,Date,SN,Port,Energy,Year,Microinversor
0,Wilkne,2025-04-06,106272403152,1,1.674,2025,Micro_01
1,Wilkne,2025-04-06,106272403152,2,1.672,2025,Micro_01
2,Wilkne,2025-04-06,106272403152,3,1.656,2025,Micro_01
3,Wilkne,2025-04-06,106272403152,4,1.663,2025,Micro_01
4,Wilkne,2025-04-06,106272403916,1,1.661,2025,Micro_02


Arquivo salvo com sucesso: files/energy_microinversores/arquivo.csv


In [8]:
import pandas as pd
import glob
import os

# Definir o caminho da pasta onde os arquivos CSV estão localizados
caminho = os.path.join("files", "energy_microinversores")
print(caminho)

# Função para obter todos os arquivos CSV na pasta
def obter_arquivos_csv(caminho):
    """Retorna uma lista de caminhos dos arquivos CSV no diretório especificado."""
    arquivos = glob.glob(os.path.join(caminho, '*.csv'))
    if not arquivos:
        print("Nenhum arquivo CSV encontrado.")
    return arquivos

# Função para ler e processar os arquivos CSV
def ler_arquivos_csv(arquivos_csv):
    """Lê todos os arquivos CSV e retorna uma lista de DataFrames."""
    lista = []
    for arquivo in arquivos_csv:
        try:
            df = pd.read_csv(arquivo, index_col=None, header=0, low_memory=False, parse_dates=['Date'])
            lista.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")
    return lista

# Salvar o DataFrame atualizado
def salvar_dataframe(df_final, caminho_novo_arquivo):
    """Salva o DataFrame atualizado em um arquivo CSV."""
    try:
        df_final.to_csv(caminho_novo_arquivo, index=False)
        print(f"Arquivo salvo com sucesso: {caminho_novo_arquivo}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")

def main():
    caminho_novo_arquivo = os.path.join(caminho, 'arquivo.csv')

    arquivos_csv = obter_arquivos_csv(caminho)
    if not arquivos_csv:
        return  # Encerra o script se não houver arquivos

    lista = ler_arquivos_csv(arquivos_csv)
    df_novo = concatenar_dataframes(lista)

    if df_novo is not None:
        df_novo = processar_dataframe(df_novo)  # Processar o DataFrame
        df_novo = adicionar_id_e_nome_microinversor(df_novo)
        # df_novo = substituir_valores_sn(df_novo)
        display(df_novo.head())
        salvar_dataframe(df_novo, caminho_novo_arquivo)

if __name__ == "__main__":
    main()

files/energy_microinversores


,Plant Name,Date,SN,Port,Energy,Year,Microinversor
0,Wilkne,2025-04-06,106272403152,1,1.674,2025,Micro_01
1,Wilkne,2025-04-06,106272403152,2,1.672,2025,Micro_01
2,Wilkne,2025-04-06,106272403152,3,1.656,2025,Micro_01
3,Wilkne,2025-04-06,106272403152,4,1.663,2025,Micro_01
4,Wilkne,2025-04-06,106272403916,1,1.661,2025,Micro_02


Arquivo salvo com sucesso: files/energy_microinversores/arquivo.csv
